In [1]:
import requests
import json
import numpy as np
import pickle
import pandas as pd
import os

In [5]:
def Collect_ids():
    import itertools
    
    ids = set()
    
    res_regions = requests.get('https://api.hh.ru/areas').json()[0]['areas']
    reg_id = set()
    
    for region in res_regions:
        reg_id.add(region['id'])
    
    experiences = set(['noExperience', 'between1And3', 'between3And6', 'moreThan6'])
    
    l = list(itertools.product(reg_id, experiences))
    for (reg, exp) in l:
        page = 0
        while True:
            response = requests.get(fr'https://api.hh.ru/vacancies?area={reg}&experience={exp}&' + 
                   'employment=full&'+
                   'schedule=fullDay&'+
                   'specialization=1&'+
                   'only_with_salary=true&'+
                   'per_page=100&'+
                   fr'page={page}')
            
            if response.status_code != 200:
                break

            json_ob = response.json()            
            if not json_ob['items']:
                break
                
            for job in json_ob['items']:
                ids.add(job['id'])
            
            page += 1
                
    return ids

In [6]:
res = Collect_ids()

In [8]:
len(res)

28651

In [10]:
with open('ids_all.pickle', 'wb') as f:
    pickle.dump(res, f)

In [64]:
def Collect_ids(text):
    import itertools
    
    ids = set()
    
    res_regions = requests.get('https://api.hh.ru/areas').json()[0]['areas']
    reg_id = set()
    
    for region in res_regions:
        reg_id.add(region['id'])
    
    experiences = set(['noExperience', 'between1And3', 'between3And6', 'moreThan6'])
    
    l = list(itertools.product(reg_id, experiences))
    for (reg, exp) in l:
        page = 0
        while True:
            response = requests.get(fr'https://api.hh.ru/vacancies?text="{text}"&search_field=name&area={reg}&experience={exp}&' + 
                   'employment=full&'+
                   'schedule=fullDay&'+
                   'specialization=1&'+
                   'only_with_salary=true&'+
                   'per_page=100&'+
                   fr'page={page}')
            
            if response.status_code != 200:
                break

            json_ob = response.json()            
            if not json_ob['items']:
                break
                
            for job in json_ob['items']:
                ids.add(job['id'])
            
            page += 1
                
    return ids

In [32]:
res_data = Collect_ids('Data')

KeyboardInterrupt: 

In [19]:
len(res_data)

82

In [58]:
with open('ids_data_2.pickle', 'wb') as f:
    pickle.dump(res_data, f)

In [65]:
res_engineer = Collect_ids('Engineer')

In [67]:
len(res_engineer)

4901

In [68]:
with open('ids_eng.pickle', 'wb') as f:
    pickle.dump(res_engineer, f)

In [33]:
res_cpp = Collect_ids('C++')

In [35]:
len(res_cpp)

1065

In [59]:
with open('ids_cpp.pickle', 'wb') as f:
    pickle.dump(res_cpp, f)

In [47]:
res_back = Collect_ids('Backend')

In [49]:
len(res_back)

273

In [60]:
with open('ids_back.pickle', 'wb') as f:
    pickle.dump(res_back, f)

In [51]:
res_front = Collect_ids('Frontend')

In [53]:
len(res_front)

327

In [61]:
with open('ids_front.pickle', 'wb') as f:
    pickle.dump(res_front, f)

In [55]:
res_front = Collect_ids('Programmer')

In [56]:
len(res_front)

5456

In [62]:
with open('ids_progr.pickle', 'wb') as f:
    pickle.dump(res_front, f)

## Continue

In [2]:
with open('ids_all.pickle', 'rb') as f:
    ids_all = pickle.load(f)
    
with open('ids_progr.pickle', 'rb') as f:
    ids_progr = pickle.load(f) 
    
with open('ids_front.pickle', 'rb') as f:
    ids_front = pickle.load(f)  

with open('ids_back.pickle', 'rb') as f:
    ids_back = pickle.load(f)  
    
with open('ids_cpp.pickle', 'rb') as f:
    ids_cpp = pickle.load(f)

with open('ids_data_2.pickle', 'rb') as f:
    ids_data = pickle.load(f)  

with open('ids_eng.pickle', 'rb') as f:
    ids_eng = pickle.load(f)

In [3]:
ids = ids_all.union(ids_progr, ids_front, ids_back, ids_cpp, ids_data, ids_eng)

In [4]:
ids_list = np.array(list(ids))

In [5]:
parts = np.array_split(ids_list, 60)

In [23]:
def collect_info(idx, d):
    resp = requests.get(fr'https://api.hh.ru/vacancies/{idx}')
    
    if resp.status_code != 200:
        return
    
    json_obj = resp.json()
    
    d['premium'].append(json_obj.get('premium', 'N/A'))
    d['name'].append(json_obj.get('name', 'N/A')) 
    d['response_letter_required'].append(json_obj.get('response_letter_required', 'N/A'))
    adress = json_obj.get('address', 'N/A')
    
    if type(adress) is dict:
        d['city'].append(adress.get('city', 'N/A'))
        d['latitude'].append(adress.get('lat', 'N/A'))
        d['longtitude'].append(adress.get('lng', 'N/A'))
        d['raw'].append(adress.get('raw', 'N/A'))
    else:
        d['city'].append('N/A')
        d['latitude'].append('N/A')
        d['longtitude'].append('N/A')
        d['raw'].append('N/A')
        
    experience = json_obj.get('experience', 'N/A')
    if type(experience) is dict:
        d['experience'].append(experience.get('name', 'N/A'))
    else:
        d['experience'].append(experience)
        
    schedule = json_obj.get('schedule', 'N/A')
    if type(schedule) is dict:
        d['schedule'].append(schedule.get('name', 'N/A'))
    else:
        d['schedule'].append(schedule)
    
    employment = json_obj.get('employment', 'N/A')
    if type(employment) is dict:
        d['employment'].append(employment.get('name', 'N/A'))
    else:
        d['employment'].append(employment)
        
    key_skills = json_obj.get('key_skills', 'N/A')
    if type(key_skills) is list:
        skills = [skill['name'] for skill in key_skills]
        d['skills'].append('_'.join(skills))
    else:
        d['skills'].append(key_skills)
        
    d['has_test'].append(json_obj.get('has_test', 'N/A')) 
    
    employer = json_obj.get('employer', 'N/A')
    if type(employer) is dict:
        d['employer'].append(employer['name'])
    else:
        d['employer'].append(employer)
        
    salary = json_obj.get('salary', 'N/A')
    if type(salary) is dict:
        d['salary_from'].append(salary.get('from', 'N/A'))
        d['salary_to'].append(salary.get('to', 'N/A'))
        d['gross'].append(salary.get('gross', 'N/A'))
        d['currency'].append(salary.get('currency', 'N/A'))
    else:
        d['salary_from'].append('N/A')
        d['salary_to'].append('N/A')
        d['gross'].append('N/A')
        d['currency'].append('N/A')

In [24]:
def create_vac_data(ids_vac):
    import pandas as pd
    from collections import defaultdict
    from tqdm import tqdm
    
    d = defaultdict(list)
    for idx in tqdm(ids_vac):
        collect_info(idx, d)
    
    df = pd.DataFrame(d)
    return df 

In [25]:
len(parts)

60

In [179]:
for (i, part) in enumerate(parts):
    
    if os.path.isfile(f'data_{i}.csv'):
        continue
    
    df_new = create_vac_data(part)
    if df_new.shape[0] < 500:
        break
    else:
        df_new.to_csv(f'data_{i}.csv', index=False, encoding='utf-8', sep='\t')
        print(f'Data part number {i} saved!')

In [2]:
def check_dims():
    import os
    import pandas as pd
    
    df_list = []
    cwd = os.getcwd()
    for file in os.listdir(cwd):
        if file.endswith(".csv"):
            try: 
                df = pd.read_csv(os.path.join(cwd, file), sep='\t')
                df_list.append(df)
                print(f'File {file} contains {df.shape[0]} rows!')
            except:
                print(f'File {file} contains 0 rows')
                
    return pd.concat(df_list)

In [3]:
df = check_dims()

File data_0.csv contains 514 rows!
File data_1.csv contains 512 rows!
File data_10.csv contains 514 rows!
File data_11.csv contains 513 rows!
File data_12.csv contains 513 rows!
File data_13.csv contains 512 rows!
File data_14.csv contains 514 rows!
File data_15.csv contains 513 rows!
File data_16.csv contains 512 rows!
File data_17.csv contains 513 rows!
File data_18.csv contains 514 rows!
File data_19.csv contains 514 rows!
File data_2.csv contains 513 rows!
File data_20.csv contains 511 rows!
File data_21.csv contains 512 rows!
File data_22.csv contains 510 rows!
File data_23.csv contains 512 rows!
File data_24.csv contains 513 rows!
File data_25.csv contains 511 rows!
File data_26.csv contains 512 rows!
File data_27.csv contains 512 rows!
File data_28.csv contains 511 rows!
File data_29.csv contains 513 rows!
File data_3.csv contains 514 rows!
File data_30.csv contains 509 rows!
File data_31.csv contains 511 rows!
File data_32.csv contains 513 rows!
File data_33.csv contains 512 ro

In [5]:
df

,premium,name,response_letter_required,city,latitude,longtitude,raw,experience,schedule,employment,skills,has_test,employer,salary_from,salary_to,gross,currency
0,False,"Консультант входящей линии ""Сети гипермаркетов...",False,Ижевск,56.840557,53.202489,"Ижевск, Красная улица, 105",Нет опыта,Полный день,Полная занятость,Деловая переписка_Грамотная речь_входящие звон...,False,Фронтлайн,NaN,32000.0,True,RUR
1,False,Менеджер-аналитик маркетплейсов,False,NaN,NaN,NaN,NaN,Нет опыта,Полный день,Полная занятость,Ведение переговоров_Поиск информации в интерне...,False,Бузенков Евгений Игоревич,50000.0,150000.0,False,RUR
2,False,Аналитик,False,Нижний Новгород,56.325759,44.015742,"Нижний Новгород, улица Нестерова, 9",От 1 года до 3 лет,Полный день,Полная занятость,Работа в команде_Работа с большим объемом инфо...,False,АНО Областное кадровое агентство,40000.0,NaN,False,RUR
3,False,Системный администратор,False,Ростов-на-Дону,47.269060,39.759832,"Ростов-на-Дону, улица 50-летия Ростсельмаша, 1/52",От 1 года до 3 лет,Полный день,Полная занятость,Asterisk_Linux_Администрирование серверов Linu...,False,Группа Компаний ЖелДорЭкспедиция,43000.0,NaN,False,RUR
4,False,Инженер - программист 1С,False,Омск,55.071494,73.296086,"Омск, проспект Губкина, 30",От 1 года до 3 лет,Полный день,Полная занятость,NaN,False,Омский каучук,60000.0,80000.0,False,RUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,False,Начальник отдела развития инфраструктуры прогр...,False,NaN,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Полная занятость,1C: Предприятие_Высшее образование_Работа в ко...,False,"Зеленая долина, ГК",150000.0,150000.0,True,RUR
509,False,Токарь,False,Рязань,54.536298,39.778209,"Рязань, Дружная, 18",От 1 года до 3 лет,Полный день,Полная занятость,Умение читать чертежи,False,"КРЗ, Многотраслевая производственная компания",45000.0,NaN,False,RUR
510,False,Сервисный инженер систем кондиционирования,False,NaN,NaN,NaN,NaN,Более 6 лет,Полный день,Полная занятость,Выездное обслуживание_Техническое обслуживание...,False,АСЦ-Климат,120000.0,160000.0,False,RUR
511,False,Системный администратор,False,NaN,NaN,NaN,NaN,От 3 до 6 лет,Полный день,Полная занятость,Администрирование сайтов_Настройка сетевых под...,False,Панацея,40000.0,NaN,False,RUR


In [192]:
df.to_csv('Final_data.csv', index = False)